### Reading data and making it clear ###

In [37]:
import pandas as pd
import numpy as np
import warnings

#------Читаем файл продаж за последний месяц ---------------------

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df = pd.read_excel('MouthSales (2).XLSX', engine="openpyxl")
    
#------Удаляем их header, сдвигаем начало df на один вниз----------

df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True)

#-----Переименовываем колонки так, чтобы там не было 2 раза'шт '--
df.columns = ['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'Поступило', 'Заказано штук', 'Сумма заказов минус комиссия WB, руб.',
       'Выкупили, шт.', 'К перечислению за товар, руб.',
       'Текущий остаток, шт.']


### Step 1: Выясняем скорость продаж ###

In [38]:
#-------Grouping by Артикул продавца -----
df_pivot = df.pivot_table(values=['Заказано штук'], index='Артикул продавца', aggfunc = 'sum').sort_values(
                                      by=['Заказано штук'], ascending=False)
#-------Average speed of buying for 30 days!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1-------
days = 14
df_pivot['Speed'] = round((df_pivot['Заказано штук'] / days), 2)


### Step 2: Выясняем остаток ###

In [39]:
#-----Остаток = Goods left ------

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df_left = pd.read_excel('left (2).xlsx', engine="openpyxl")
    
df_left_pivot = df_left.pivot_table(
    values=['Итого по складам', 'Товары в пути до клиента', 'Товары в пути от клиента'], 
    index='Артикул продавца', 
    aggfunc = 'sum')

df_left_pivot['Left'] = df_left_pivot['Итого по складам']


### Step 3: Узнаем, сколько нужно поставить и какой продукт продается слабо ###

In [40]:
#----------------Combining two tables -----------------
result = pd.merge(df_pivot, df_left_pivot, on="Артикул продавца")
result.drop(columns = ['Заказано штук', 'Итого по складам', 'Товары в пути до клиента', 'Товары в пути от клиента'], axis = 1, 
            inplace = True) 

In [41]:
#------------- How much to deliver so it is okey for next 60 days  --------
result['To deliver'] = result['Speed'] * 28 + 10 - result['Left']

#------------- Find a week product ----------------------------------------
result['Weak product'] = np.where(result['Speed'] > 0.5, False, True)
result.sort_values(by=['To deliver'], ascending=False, inplace=True)

In [42]:
result.to_excel("To delivernew2.xlsx") 